<a href="https://colab.research.google.com/github/kinjaljoshi/ma_rag_components/blob/master/indexing_metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Following code shows storing table context and api context in vector DB

In [ ]:
!pip install langchain faiss-cpu huggingface-hub langchain_community

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
import os

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# === Step 1: Create DB Embeddings
sales_order = Document(
    page_content="""
    Table: sales_orders Description: Contains all sales order transactions with customer and amount.
    Columns:
    so_id : 'Unique Sales Order Identifier',
    customer_id : 'Customer Identifier' ,
    total_amount : 'Sales Order Amount',
    order_date : 'Sales Order Date'
    """,
    metadata={
        "object_name": "sales_orders",
        "object_type": "table",
        "access_method": "sql",
        "table_name": "sales.sales_orders"
    }
)


sales_order_status = Document(
    page_content="""
    Table: sales_order_status Description: Tracks the latest status of each sales order (e.g., shipped, invoiced).
    Columns:
    so_id : 'Unique Sales Order Identifier',
    status : 'Sales Order Status'
    """,
    metadata={
        "object_name": "sales_order_status",
        "object_type": "table",
        "access_method": "sql",
        "source": "bigquery",
        "table_name": "sales.sales_order_status"
    }
)

# === Step 2: Create API Embeddings

purchase_order = Document(
    page_content="""
    API Object: Purchase Order
    Endpoint: /api/purchase-orders
    Fields: po_id (string), vendor_id (string), amount (float), status (string)
    Description: Retrieves purchase orders from the ERP system.
    Sample response: {"po_id": "PO123", "vendor_id": "VEND001", "amount": 10000, "status": "approved"}
    """,
    metadata={
        "object_name": "purchase_orders",
        "object_type": "api",
        "access_method": "api",
        "source": "erp",
        "endpoint": "/api/purchase-orders"
    }
)

purchase_order_lines = Document(
    page_content="""
    API Object: Purchase Order Line Items
    Endpoint: /api/purchase-orders/line-items
    Fields: po_id (string), item_id (string), sku (string), quantity (int), unit_price (float)
    Description: Provides line-item level detail for each purchase order.
    Sample response: {"po_id": "PO123", "item_id": "ITEM999", "sku": "SKU789", "quantity": 4, "unit_price": 2500}
    """,
    metadata={
        "object_name": "purchase_order_lines",
        "object_type": "api",
        "access_method": "api",
        "source": "erp",
        "endpoint": "/api/purchase-orders/line-items"
    }
)

# === Step 3: Generate Embeddings and Store in FAISS ===

documents = [
    sales_order,
    sales_order_status,
    purchase_order,
    purchase_order_lines
]

print("Generating embeddings...")
faiss_vectorstore = FAISS.from_documents(documents, embedding_model)

# === Step 4: Save FAISS Index Locally ===
faiss_vectorstore.save_local("faiss_embeddings")

print("Successfully generated embeddings")


Generating embeddings...
Successfully generated embeddings


In [16]:
faiss_vectorstore = FAISS.load_local("/content/faiss_embeddings", embedding_model, allow_dangerous_deserialization=True)
query = "Get purchase order details"
results = faiss_vectorstore.similarity_search(query, k=2)

for doc in results:
    print("\n--- Match ---")
    print("Object Name:", doc.metadata["object_name"])
    print("Object Type:", doc.metadata["object_type"])
    print(doc.page_content[:1000])



--- Match ---
Object Name: purchase_orders
Object Type: api

    API Object: Purchase Order
    Endpoint: /api/purchase-orders
    Fields: po_id (string), vendor_id (string), amount (float), status (string)
    Description: Retrieves purchase orders from the ERP system.
    Sample response: {"po_id": "PO123", "vendor_id": "VEND001", "amount": 10000, "status": "approved"}
    

--- Match ---
Object Name: purchase_order_items
Object Type: api

    API Object: Purchase Order Line Items
    Endpoint: /api/purchase-orders/line-items
    Fields: po_id (string), item_id (string), sku (string), quantity (int), unit_price (float)
    Description: Provides line-item level detail for each purchase order.
    Sample response: {"po_id": "PO123", "item_id": "ITEM999", "sku": "SKU789", "quantity": 4, "unit_price": 2500}
    


In [17]:
query = "Get Sales order details"
results = faiss_vectorstore.similarity_search(query, k=2)

for doc in results:
    print("\n--- Match ---")
    print("Object Name:", doc.metadata["object_name"])
    print("Object Type:", doc.metadata["object_type"])
    print(doc.page_content[:1000])



--- Match ---
Object Name: sales_order_status
Object Type: table

    Table: sales_order_status Description: Tracks the latest status of each sales order (e.g., shipped, invoiced).
    Columns: 
    so_id : 'Unique Sales Order Identifier',
    status : 'Sales Order Status'    
    

--- Match ---
Object Name: sales_orders
Object Type: table

    Table: sales_orders Description: Contains all sales order transactions with customer and amount.
    Columns: 
    so_id : 'Unique Sales Order Identifier', 
    customer_id : 'Customer Identifier' ,
    total_amount : 'Sales Order Amount',
    order_date : 'Sales Order Date'
    
